In [23]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import scipy.stats as stats

In [24]:
fluence = pd.read_csv('reinserted_thermal_fluence.csv')

In [25]:
fluence

,# timestamp,cesium_val_0,cesium_val_1,cesium_val_2,cesium_val_3,cesium_val_4,cesium_val_5,cesium_val_6,cesium_val_7,cesium_val_8,...,fluence_val_803,fluence_val_804,fluence_val_805,fluence_val_806,fluence_val_807,fluence_val_808,fluence_val_809,fluence_val_810,fluence_val_811,fluence_val_812
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,3.474796e-08,1.707146e-07,7.250270e-07,1.518698e-07,2.655534e-07,1.096870e-07,4.931048e-07,5.913665e-08,7.028061e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2.246938e-06,2.872698e-06,2.737213e-06,3.099643e-06,2.319968e-06,2.971253e-06,4.813968e-07,1.661822e-06,3.443706e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,4.538420e-06,4.424400e-06,3.635429e-06,5.124641e-06,4.557323e-06,5.301739e-06,3.826000e-06,6.284903e-06,5.364588e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,191.0,0.000000e+00,5.054584e-05,5.790360e-05,4.756137e-05,0.000000e+00,0.000000e+00,6.990133e-05,0.000000e+00,4.825021e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,192.0,5.011145e-05,8.686642e-05,0.000000e+00,4.515687e-05,0.000000e+00,0.000000e+00,4.304290e-05,4.031293e-05,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,193.0,0.000000e+00,0.000000e+00,4.521334e-05,7.796768e-05,0.000000e+00,4.886403e-05,0.000000e+00,0.000000e+00,6.935493e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,194.0,0.000000e+00,4.109691e-05,0.000000e+00,9.213985e-05,7.296321e-05,0.000000e+00,0.000000e+00,3.900867e-05,4.373237e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def bin_names(arr):
    arr_names = []
    for i in range(len(arr)):
        arr[i] = np.format_float_scientific(arr[i], unique=False, precision=2)
        if i+2 > len(arr):
            break
        else:
            arr[i+1] = np.format_float_scientific(arr[i+1], unique=False, precision=2)
            arr_names.append(str([str(arr[i]) + ':' + str(arr[i+1])]))
    return arr_names

In [27]:
def fluence_bins(fluence, fluence_col_start, initial_burn_up):
    fluence_only = fluence.iloc[initial_burn_up:,fluence_col_start:]
    timestamp = fluence[['# timestamp']]
    fluence_t = pd.concat([timestamp, fluence_only], axis=1)
    # Read data
    Tbl = fluence_t

    # Extract data
    Xvals = Tbl.iloc[:,1:-1].to_numpy()
    Yvals = Tbl.iloc[:,-1].to_numpy()
    timestamps = Tbl.iloc[:,0].to_numpy()
    nRows, nCols = Xvals.shape
    #Xavg = np.mean(Xvals, axis=1)

    # define bins
    binMax = Tbl.iloc[:,1:].max().max()
    binInt = binMax/200
    hEdges = np.arange(0, binMax+binInt, binInt)
    nBins = len(hEdges)-1

    # fill histogram matrix
    nCountsX = np.zeros((nRows, nBins), dtype=np.uint16)
    for n in range(nRows):
        nCountsX[n,:], _ = np.histogram(Xvals[n,:], hEdges)
        
    counts_df = pd.DataFrame(nCountsX)
    hEdges_name = bin_names(hEdges)
    counts_df.index = timestamps
    counts_df.index.name = 't'
    counts_df.columns = hEdges_name
    return counts_df

In [33]:
def isotope_bins(data, isotope_col_start, isotope_col_end, initial_burnup, num_bins = 200):
    isotope_only = data.iloc[initial_burnup:,isotope_col_start:isotope_col_end]
    timestamp = data[['# timestamp']].iloc[initial_burnup:]
    Tbl = pd.concat([timestamp, isotope_only], axis=1)

    # Extract data
    Xvals = Tbl.iloc[:,1:-1].to_numpy()
    Yvals = Tbl.iloc[:,-1].to_numpy()
    timestamps = Tbl.iloc[:,0].to_numpy()
    nRows, nCols = Xvals.shape
    #Xavg = np.mean(Xvals, axis=1)

    # define bins
    binMax = Tbl.iloc[:,1:].max().max()
    binInt = binMax/num_bins
    hEdges = np.arange(0, binMax+binInt, binInt)
    nBins = len(hEdges)-1

    # fill histogram matrix
    nCountsX = np.zeros((nRows, nBins), dtype=np.uint16)
    for n in range(nRows):
        nCountsX[n,:], _ = np.histogram(Xvals[n,:], hEdges)
    
    counts_df = pd.DataFrame(nCountsX)
    hEdges_name = bin_names(hEdges)
    counts_df.index = timestamps
    counts_df.index.name = 't'
    counts_df.columns = hEdges_name
    return counts_df

In [42]:
def concat_isotopes(isotopes,isotope_col_start, isotope_col_end, initial_burnup, num_bins = 200):
    prev_table = False
    df = 0
    for i in range(len(isotopes)):
        if prev_table == False:
            df = isotope_bins(isotopes[i],isotope_col_start[i], isotope_col_end[i], initial_burnup, num_bins)
        else:
            df2 = isotope_bins(isotopes[i],isotope_col_start[i], isotope_col_end[i], initial_burnup, num_bins)
            pd.merge(df, df2, left_index=True, right_index=True)
    return df

In [34]:
fluence_binned = isotope_bins(fluence, 814, 1627, 50)
cesium_binned = isotope_bins(fluence, 50, 814, 50)

In [35]:
fluence_binned

,['0.0:7.21e+18'],['7.21e+18:1.44e+19'],['1.44e+19:2.16e+19'],['2.16e+19:2.88e+19'],['2.88e+19:3.6e+19'],['3.6e+19:4.32e+19'],['4.32e+19:5.04e+19'],['5.04e+19:5.77e+19'],['5.77e+19:6.49e+19'],['6.49e+19:7.21e+19'],...,['1.38e+21:1.38e+21'],['1.38e+21:1.39e+21'],['1.39e+21:1.4e+21'],['1.4e+21:1.41e+21'],['1.41e+21:1.41e+21'],['1.41e+21:1.42e+21'],['1.42e+21:1.43e+21'],['1.43e+21:1.43e+21'],['1.43e+21:1.44e+21'],['1.44e+21:1.45e+21']
t,,,,,,,,,,,,,,,,,,,,,
50.0,319,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
51.0,311,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52.0,294,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53.0,296,0,0,0,0,0,0,0,0,0,...,4,0,2,1,0,0,0,0,0,0
54.0,280,0,0,0,0,0,0,0,0,0,...,2,3,2,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191.0,355,0,0,0,0,0,0,0,0,0,...,3,1,1,2,0,0,0,0,0,0
192.0,347,0,0,0,0,0,0,0,0,0,...,1,0,1,1,2,0,0,0,0,0
193.0,344,0,0,0,0,0,0,0,0,0,...,2,0,0,1,2,0,0,0,0,0


In [36]:
cesium_binned

,['0.0:5.33e-07'],['5.33e-07:1.07e-06'],['1.07e-06:1.6e-06'],['1.6e-06:2.13e-06'],['2.13e-06:2.67e-06'],['2.67e-06:3.2e-06'],['3.2e-06:3.73e-06'],['3.73e-06:4.27e-06'],['4.27e-06:4.8e-06'],['4.8e-06:5.33e-06'],...,['0.000101:0.000102'],['0.000102:0.000102'],['0.000102:0.000103'],['0.000103:0.000103'],['0.000103:0.000104'],['0.000104:0.000104'],['0.000104:0.000105'],['0.000105:0.000106'],['0.000106:0.000106'],['0.000106:0.000107']
t,,,,,,,,,,,,,,,,,,,,,
50.0,272,0,0,0,0,0,0,0,0,0,...,0,2,0,2,3,3,2,1,0,2
51.0,243,0,0,0,0,0,0,0,0,0,...,4,0,2,3,3,1,0,1,0,0
52.0,236,0,0,0,0,0,0,0,0,0,...,4,3,3,2,2,0,2,1,1,0
53.0,222,0,0,0,0,0,0,0,0,0,...,1,0,2,3,2,2,2,3,3,2
54.0,244,0,0,0,0,0,0,0,0,0,...,1,4,2,2,1,5,2,2,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191.0,286,0,0,0,0,0,0,0,0,0,...,1,2,2,7,3,2,4,3,5,4
192.0,275,0,0,0,0,0,0,0,0,0,...,3,1,4,1,6,3,2,2,2,4
193.0,276,0,0,0,0,0,0,0,0,0,...,3,2,3,3,4,5,6,5,2,2


In [37]:
fluence_binned.to_csv('fluence.csv')
cesium_binned.to_csv('cesium.csv')